In [31]:
from typing import Any
import os
from unstructured.partition.pdf import partition_pdf
import os
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
import pytesseract
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI, AsyncAzureOpenAI
import uuid
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma

In [12]:
# Configure environment variables  
load_dotenv(dotenv_path='./.env')

True

In [2]:
#print(os.getenv('PATH'))
#os.environ["PATH"] = os.getenv('PATH') + ';' + 'C:\\Program Files\\poppler-24.02.0\\Library\\bin\\'
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [3]:
bofa2018 = "./Data/BOFA/BofA 2018.pdf"
# Get elements
rawPdfElements = partition_pdf(
    filename=bofa2018,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
)

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

In [5]:
import base64
text_elements = []
table_elements = []
image_elements = []

# Function to encode images
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

for element in rawPdfElements:
    if 'CompositeElement' in str(type(element)):
        text_elements.append(element)
    elif 'Table' in str(type(element)):
        table_elements.append(element)

table_elements = [i.text for i in table_elements]
text_elements = [i.text for i in text_elements]

# Tables
print(len(table_elements))

# Text
print(len(text_elements))

247
398


In [8]:
# for image_file in os.listdir(output_path):
#     if image_file.endswith(('.png', '.jpg', '.jpeg')):
#         image_path = os.path.join(output_path, image_file)
#         encoded_image = encode_image(image_path)
#         image_elements.append(encoded_image)
# print(len(image_elements))

In [35]:
client = AzureOpenAI(
    api_key=os.getenv('OpenAiWestUsKey'),  
    api_version=os.getenv('OpenAiGpt4vVersion'),
    #base_url=f"{os.getenv('OpenAiWestUsEp')}openai/deployments/{os.getenv('OpenAiGpt4v')}/extensions",
    azure_endpoint=os.getenv('OpenAiWestUsEp'),
)
embeddings = AzureOpenAIEmbeddings(azure_endpoint=os.getenv('OpenAiEndPoint'), azure_deployment=os.getenv('OpenAiEmbedding'), 
                                   api_key=os.getenv('OpenAiKey'), openai_api_type="azure")

In [19]:
llm = AzureChatOpenAI(
                openai_api_base=os.getenv('OpenAiEndPoint'),
                openai_api_version=os.getenv('OpenAiVersion'),
                deployment_name=os.getenv('OpenAiChat'),
                openai_api_key=os.getenv('OpenAiKey'),
                openai_api_type="azure",
                max_tokens=1024)

C:\Users\astalati\AppData\Roaming\Python\Python311\site-packages\langchain\chat_models\azure_openai.py:156: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://dataaiapim.azure-api.net to https://dataaiapim.azure-api.net/openai.
  warnings.warn(
C:\Users\astalati\AppData\Roaming\Python\Python311\site-packages\langchain\chat_models\azure_openai.py:163: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\astalati\AppData\Roaming\Python\Python311\site-packages\langchain\chat_models\azure_openai.py:171: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource

In [26]:
def summarizeText(text):
    prompt = f"Summarize the following text:\n\n{text}\n\nSummary:"
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content

# Function for table summaries
def summarizeTable(table):
    prompt = f"Summarize the following table:\n\n{table}\n\nSummary:"
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content

# Function to add documents to the retriever
def addDocumentToRetriever(retriever, idKey, summaries, originalContents):
    doc_ids = [str(uuid.uuid4()) for _ in summaries]
    summary_docs = [
        Document(page_content=s, metadata={idKey: doc_ids[i]})
        for i, s in enumerate(summaries)
    ]
    retriever.vectorstore.add_documents(summary_docs)
    retriever.docstore.mset(list(zip(doc_ids, originalContents)))

In [21]:
# Processing text elements with feedback and sleep
textSummaries = []
for i, te in enumerate(text_elements):
    summary = summarizeText(te)
    textSummaries.append(summary)
    print(f"{i + 1}th element of texts processed.")

1th element of texts processed.
2th element of texts processed.
3th element of texts processed.
4th element of texts processed.
5th element of texts processed.
6th element of texts processed.
7th element of texts processed.
8th element of texts processed.
9th element of texts processed.
10th element of texts processed.
11th element of texts processed.
12th element of texts processed.
13th element of texts processed.
14th element of texts processed.
15th element of texts processed.
16th element of texts processed.
17th element of texts processed.
18th element of texts processed.
19th element of texts processed.
20th element of texts processed.
21th element of texts processed.
22th element of texts processed.
23th element of texts processed.
24th element of texts processed.
25th element of texts processed.
26th element of texts processed.
27th element of texts processed.
28th element of texts processed.
29th element of texts processed.
30th element of texts processed.
31th element of tex

In [22]:
# Processing table elements with feedback and sleep
tableSummaries = []
for i, te in enumerate(table_elements):
    summary = summarizeTable(te)
    tableSummaries.append(summary)
    print(f"{i + 1}th element of tables processed.")

1th element of tables processed.
2th element of tables processed.
3th element of tables processed.
4th element of tables processed.
5th element of tables processed.
6th element of tables processed.
7th element of tables processed.
8th element of tables processed.
9th element of tables processed.
10th element of tables processed.
11th element of tables processed.
12th element of tables processed.
13th element of tables processed.
14th element of tables processed.
15th element of tables processed.
16th element of tables processed.
17th element of tables processed.
18th element of tables processed.
19th element of tables processed.
20th element of tables processed.
21th element of tables processed.
22th element of tables processed.
23th element of tables processed.
24th element of tables processed.
25th element of tables processed.
26th element of tables processed.
27th element of tables processed.
28th element of tables processed.
29th element of tables processed.
30th element of tables 

In [34]:
embeddings

AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001FB133FB310>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001FADBCFB310>, model='text-embedding-ada-002', deployment='embedding', openai_api_version='2023-05-15', openai_api_base=None, openai_api_type='azure', openai_proxy='', embedding_ctx_length=8191, openai_api_key='1361532858cb428c8da412a73105de78', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=16, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, azure_endpoint='https://dataaiapim.azure-api.net', azure_ad_token=None, azure_ad_token_provider=None, validate_base_url=True)

In [37]:
# Initialize the vector store and storage layer
vectorStore = Chroma(collection_name="summaries", embedding_function=AzureOpenAIEmbeddings(azure_endpoint=os.getenv('OpenAiEndPoint'), azure_deployment=os.getenv('OpenAiEmbedding'), 
                                   api_key=os.getenv('OpenAiKey'), openai_api_type="azure"))
store = InMemoryStore()
idKey = "doc_id"

# Initialize the retriever
retriever = MultiVectorRetriever(vectorstore=vectorStore, docstore=store, id_key=idKey)


In [38]:
# Add text summaries
addDocumentToRetriever(retriever, idKey, textSummaries, text_elements)
# Add table summaries
addDocumentToRetriever(retriever, idKey, tableSummaries, table_elements)

In [40]:
# We can retrieve this table
retriever.get_relevant_documents(
    "What is the trading revenue from sales and equities?"
)

['(Dollars in millions) Sales and trading revenue 2018 2017 Fixed-income, currencies and commodities $ 8,186 $ Equities 4,876 Total sales and trading revenue $ 13,062 $ Sales and trading revenue, excluding net DVA (3) Fixed-income, currencies and commodities $ 8,328 $ Equities 4,896 Total sales and trading revenue, excluding net DVA $ 13,224 $ 8,657 4,120 12,777 9,051 4,154 13,205',
 'Sales and Trading Revenue (1, 2)\n\nThe return on average allocated capital was11 percent, up from 9 percent, reflecting higher net income. For more information on capital allocated to the business segments, see Business Segment Operations on page 30.\n\nSales and Trading Revenue Sales and trading revenue includes unrealized and realized gains and losses on trading and other assets, net interest income, and fees primarily from commissions on equity securities. Sales and trading revenue is segregated into fixed-income (government debt obligations, investment and non-investment grade corporate debt obligati

In [41]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

template = """Answer the question based only on the following context, which can include text, images and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [42]:
chain.invoke(
     "What is the trading revenue from sales and equities?"
)

'The trading revenue from sales and equities is $4,876 million.'

In [43]:
chain.invoke("What is the percentage change in cash and cash equivalents from 2017 and 2018?")

'The percentage change in cash and cash equivalents from 2017 to 2018 is 13%.'

In [44]:
chain.invoke("Was trading account liabilities positive or negative from 2017 and 2018 and by how much?")

'To determine whether trading account liabilities were positive or negative in 2017 and 2018, we need to analyze the information provided. Unfortunately, the given context does not include any specific data or figures related to trading account liabilities. Therefore, we cannot answer the question based on the provided context.'

In [49]:
chain.invoke("What are the five segments for operations, give details in bulleted format")

'The five segments for operations are:\n\n- Consumer Banking: Offers a range of credit, banking, and investment products and services to consumers and small businesses. Product offerings include savings accounts, CDs, checking accounts, residential mortgages, and loans.\n- Global Wealth & Investment Management (GWIM): Provides tailored financial solutions to clients with over $250,000 in total investable assets. Services include wealth management, brokerage, banking, retirement products, and specialty asset management.\n- Global Banking: Offers lending-related products and services, working capital management, treasury solutions, and investment banking products. Clients include middle-market companies, commercial real estate firms, global corporations, financial institutions, and mid-sized U.S.-based businesses.\n- Global Markets: Provides sales and trading services, research services, market-making, financing, and risk management products to institutional clients. Covers fixed-income,

In [51]:
chain.invoke("What is the net interest income from deposits in 2018 for Consumer banking business?")

'The net interest income from deposits in 2018 for the Consumer Banking business is not provided in the given context.'